In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
    'lhc_per': 'vid|a3d85efe963899fee2b5',
    '_gid': 'GA1.3.289911229.1685154797',
    '_ga': 'GA1.1.2075372866.1684772919',
    '_ga_3C534XGNC2': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_Q33BVTTDL1': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_QHEB12PC6D': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28F695C90B262BFC00B79BBD6CC1; lhc_per=vid|a3d85efe963899fee2b5; _gid=GA1.3.289911229.1685154797; _ga=GA1.1.2075372866.1684772919; _ga_3C534XGNC2=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_Q33BVTTDL1=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_QHEB12PC6D=GS1.1.1685154797.4.1.1685154882.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies, headers=headers)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [5]:
files = glob('pages-ums/*.json')
len(files)

2717

In [6]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [7]:
filtered_pdfs = []
for p in tqdm(pdfs):
#     if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
#         continue
#     if not p.endswith('.pdf'):
#         continue
#     if 'ade.sagepub.com' in p:
#         continue
    
#     if p.startswith('/jspui'):
#         p = 'https://oarep.usim.edu.my' + p
        
    if not p.startswith('http'):
        continue
        
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|████████████████████████████████| 54658/54658 [00:00<00:00, 4180460.40it/s]


54611

In [9]:
# filtered_pdfs

In [10]:
ums = filtered_pdfs

In [11]:
# !rm -rf usim
!mkdir ums

In [12]:
max_worker = 1

ums = sorted(list(ums))
for i in tqdm(range(1, len(ums), max_worker)):
    urls = [(url, os.path.join('ums', f'{i}-{no}.pdf')) for no, url in enumerate(ums[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  0%|                                                 | 0/54610 [00:00<?, ?it/s]

0 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S0308814613000824/1-s2.0-S0308814613000824-main.pdf?_tid=d4f13cc4-df01-11e2-80fa-00000aacb35e&acdnat=1372321153_5aedaef90504b94eb1b60517305a6e93 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7282405910>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S0308814613000824/1-s2.0-S0308814613000824-main.pdf?_tid=d4f13cc4-df01-11e2-80fa-00000aacb35e&acdnat=1372321153_5aedaef90504b94eb1b60517305a6e93 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f72824050a0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S0308814613000824/1-s2.0-S0308814613000824-main.pdf?_tid=d4f13cc4-df01-11e2-80fa-00000aacb35e&acdnat=137232115

  0%|                                      | 1/54610 [00:01<29:41:54,  1.96s/it]

0 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S1877042813019563/1-s2.0-S1877042813019563-main.pdf?_tid=df987198-fdb3-11e3-9853-00000aacb35d&acdnat=1403843654_c85ba40204a0212648aca8343da73e41 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7282405910>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S1877042813019563/1-s2.0-S1877042813019563-main.pdf?_tid=df987198-fdb3-11e3-9853-00000aacb35d&acdnat=1403843654_c85ba40204a0212648aca8343da73e41 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7282405760>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPConnectionPool(host='ac.els-cdn.com', port=80): Max retries exceeded with url: /S1877042813019563/1-s2.0-S1877042813019563-main.pdf?_tid=df987198-fdb3-11e3-9853-00000aacb35d&acdnat=140384365

  0%|                                    | 11/54610 [02:19<579:37:16, 38.22s/it]

...

100%|█████████████████████████████████▊| 54342/54610 [15:50:45<17:49,  3.99s/it]

0 HTTPSConnectionPool(host='scientificarray.org', port=443): Max retries exceeded with url: /ijgt/wp-content/uploads/2022/04/IJGTV8A1-Wong-1.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f728243c760>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPSConnectionPool(host='scientificarray.org', port=443): Max retries exceeded with url: /ijgt/wp-content/uploads/2022/04/IJGTV8A1-Wong-1.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f72801c4dc0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPSConnectionPool(host='scientificarray.org', port=443): Max retries exceeded with url: /ijgt/wp-content/uploads/2022/04/IJGTV8A1-Wong-1.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f728243c940>: Failed to establish a new connection: [Errno -2] Name or service not known'))


100%|█████████████████████████████████▉| 54580/54610 [15:57:43<00:54,  1.83s/it]

0 HTTPSConnectionPool(host='www.thaiscience.info', port=443): Max retries exceeded with url: /Journals/Article/SONG/10993068.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f72803ceb50>: Failed to establish a new connection: [Errno 110] Connection timed out'))
1 HTTPSConnectionPool(host='www.thaiscience.info', port=443): Max retries exceeded with url: /Journals/Article/SONG/10993068.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f728036bb50>: Failed to establish a new connection: [Errno 110] Connection timed out'))
2 HTTPSConnectionPool(host='www.thaiscience.info', port=443): Max retries exceeded with url: /Journals/Article/SONG/10993068.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f728036d940>: Failed to establish a new connection: [Errno 110] Connection timed out'))


100%|██████████████████████████████████| 54610/54610 [16:04:42<00:00,  1.06s/it]
